**Hi everybody!!! This notebook is the work of a couple of weeks with nnUnet software, which was the BRATS'20 winner. I have been working with the BRATS'21 task1 data, and also competition data for brain tumour segmentation. I really expect what could be helpfull for you!**

- My first approximation was training with nnUnet data my own model, but due to kaggle GPU limits, when I was trying to train 2D, and 3D-FullRes models, I couldn't finish because it took more than 9 hours. Neverless, I used the pretrained models, and here you can see the results:

# 1.- Installing nnUNET and import libs. 

In [ ]:
!git clone https://github.com/MIC-DKFZ/nnUNet.git
!git clone https://github.com/NVIDIA/apex
!pip install -e ./nnUNet
!pip install --upgrade git+https://github.com/nanohanno/hiddenlayer.git@bugfix/get_trace_graph#egg=hiddenlayer

In [ ]:
###Libraries and imports
import numpy as np
import math
import random
import os
import shutil
import gzip
import shutil
import glob
import gc
import cv2 as cv
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd
import tarfile
import PIL
import scipy.misc
import skimage
import nibabel as nib
import matplotlib.pyplot as plt
import sys

from PIL import Image
from tqdm import tqdm
from tensorflow import keras
from keras.models import Model, load_model
from keras.layers import Input ,BatchNormalization , Activation 
from keras.layers.convolutional import Conv2D, UpSampling2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import optimizers 
from keras import backend as K
from sklearn.model_selection import train_test_split
from skimage.morphology import ball, disk, dilation, binary_erosion, remove_small_objects, erosion, closing, reconstruction, binary_closing
from skimage.measure import label,regionprops, perimeter
from skimage.morphology import binary_dilation, binary_opening
from skimage.filters import roberts, sobel
from skimage import measure, feature
from skimage.segmentation import clear_border
from skimage import data
from skimage.io import imread
from scipy import ndimage as ndi
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from glob import glob
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
import SimpleITK as sitk
from skimage.transform import resize
import os
import sys 
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import pydicom
from os.path import join
from fastai.vision.all import *
DEVICE = "GPU"



# 2.- Download Brats'20 pretrained model and prediction with BRATS'21 dataset

- Download pretrained Task001_BrainTumour, and setting up the nnUnet environment.

In [ ]:

os.mkdir('./nnUNet_raw_data_base')
os.mkdir('./nnUNet_raw_data_base/nnUNet_raw_data')
os.mkdir('./RESULTS_FOLDER')
os.environ['nnUNet_raw_data_base'] = './nnUNet_raw_data_base/nnUNet_raw_data'
os.environ['RESULTS_FOLDER'] = './RESULTS_FOLDER'
os.environ['nnUNet_preprocessed'] = './nnUNet_preprocessed'
!nnUNet_download_pretrained_model Task001_BrainTumour

- In order to show how it work for BRATS'21, I will execute one example case.

-- Important: The data as i told you should be structurated as follow, and as you can see in brats2021dataset:
in the same study, you have to rename the files with 00000, 00001, 00002 and 00003 depending if it is an MRI type FLAIR, T1w, T1wCE of T2 respectively, and it should be in .gz format.

In [ ]:
!mkdir -p ./RESULTS_FOLDER/nnUNet/2d/image_raw
!cp ../input/brats2021dataset/nnUNet_raw_data_base/nnUNet_raw_data/Task101_BrainTumour/imagesTs/BRATS_188_0000.nii ./RESULTS_FOLDER/nnUNet/2d/image_raw/BRATS_188_0000.nii.gz
!cp ../input/brats2021dataset/nnUNet_raw_data_base/nnUNet_raw_data/Task101_BrainTumour/imagesTs/BRATS_188_0001.nii ./RESULTS_FOLDER/nnUNet/2d/image_raw/BRATS_188_0001.nii.gz
!cp ../input/brats2021dataset/nnUNet_raw_data_base/nnUNet_raw_data/Task101_BrainTumour/imagesTs/BRATS_188_0002.nii ./RESULTS_FOLDER/nnUNet/2d/image_raw/BRATS_188_0002.nii.gz
!cp ../input/brats2021dataset/nnUNet_raw_data_base/nnUNet_raw_data/Task101_BrainTumour/imagesTs/BRATS_188_0003.nii ./RESULTS_FOLDER/nnUNet/2d/image_raw/BRATS_188_0003.nii.gz
!nnUNet_predict -i ./RESULTS_FOLDER/nnUNet/2d/image_raw -o ./RESULTS_FOLDER/nnUNet/2d/ -t "001" -tr nnUNetTrainerV2 -m 2d


In [ ]:
!ls ./RESULTS_FOLDER/nnUNet/2d/image_raw/

In [ ]:
path_raw="../input/brats2021dataset/nnUNet_raw_data_base/nnUNet_raw_data/Task101_BrainTumour/imagesTs/BRATS_188_0000.nii"
path="./RESULTS_FOLDER/nnUNet/2d/"

plt.figure(figsize=(12,6))
plt.subplot(121)
flair_nib = nib.load(path_raw)
flair_nib_array = flair_nib.get_fdata()
plt.imshow(flair_nib_array[:,:,flair_nib_array.shape[2]//2])
plt.subplot(122)
flair_nib = nib.load(path+"BRATS_188.nii.gz")
flair_nib_array = flair_nib.get_fdata()
plt.imshow(flair_nib_array[:,:,flair_nib_array.shape[2]//2])

# 3.- Prediction of RSNA-MICCAI challenge code, transform size and format previously, and show the predictions

- Now that you can see that it works for BRAT'S 21, lets see how to manage with competition data:
-- You have to move all the data to nii.gz format, and also move to this shape configuration (240,240,155). So, the way that I have used this data, first resize to 240x240, and secondly, if there are enought .dcm images, it is no problem, I can cut in 155, and get the 240x240x155, but it is not the case, you can create empty slices.

In [ ]:
!mkdir -p ./test/train/
!mkdir -p ./test/test/

####Data test
train_path="../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/"
studi_id=["00000","00002","00003"]
#############

####Data PRD TRAIN
from glob import glob

samples_train = ("../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/*")
path_train=glob(samples_train)
train_path=""
studi_id=path_train

mri_types = ["FLAIR","T1w","T1wCE","T2w"]
pixel_size_h=240
pixel_size_w=240
inner_count=0

def dicom2nifti(image_dir, out_dir, save=True):
    "given a dicom directory, loads them into single file and can save it as .nii file"
    reader = sitk.ImageSeriesReader()
    reader.LoadPrivateTagsOn()
    filenamesDICOM = reader.GetGDCMSeriesFileNames(str(image_dir))
    reader.SetFileNames(filenamesDICOM)
    img = reader.Execute()
    img = sitk.Cast(img, sitk.sitkFloat32)
    
    if save:
        sitk.WriteImage(img, f'{out_dir}/{image_dir.parent.name}.nii')
    else:
        return img

def resample_nifti(image_dir, ref_image, fn, save=True):
    "resample using a reference image"

    image = sitk.ReadImage(str(image_dir), sitk.sitkFloat32)
    
    initial_transform = sitk.CenteredTransformInitializer(ref_image, 
                                                          image, 
                                                          sitk.Euler3DTransform(), 
                                                          sitk.CenteredTransformInitializerFilter.GEOMETRY)

    resampler = sitk.ResampleImageFilter()
    resampler.SetReferenceImage(ref_image)
    resampler.SetInterpolator(sitk.sitkLinear)
    resampler.SetTransform(initial_transform)
    resampler.SetOutputSpacing(ref_image.GetSpacing())
    resampler.SetSize((ref_image.GetSize()))
    resampler.SetOutputDirection(ref_image.GetDirection())
    resampler.SetOutputOrigin(ref_image.GetOrigin())
    resampler.SetDefaultPixelValue(image.GetPixelIDValue())
    resamped_image = resampler.Execute(image)
    
    if save:
        sitk.WriteImage(resamped_image, fn)

    return resamped_image

ref_image = sitk.ReadImage('../input/sri24-dataset/sri24/spgr.nii', sitk.sitkFloat32)
!mkdir -p ./tmp/T1w
!mkdir -p ./tmp/T1wCE
!mkdir -p ./tmp/T2w
!mkdir -p ./tmp/FLAIR

for c in tqdm(studi_id):
    path=join(train_path,c)
    samples = [Path(c)]
    path_train_t2w, path_train_t1wce,path_train_t1w,path_train_flair = [],[],[],[]
    for each in samples:
        path_train_t2w.append(each.ls()[0])
        path_train_t1wce.append(each.ls()[1])
        path_train_t1w.append(each.ls()[2])
        path_train_flair.append(each.ls()[3])
    for fn in path_train_t1w: dicom2nifti(fn, "./tmp/T1w/")
    for fn in path_train_t1wce: dicom2nifti(fn, "./tmp/T1wCE/")
    for fn in path_train_t1w: dicom2nifti(fn, "./tmp/T2w/")
    for fn in path_train_flair: dicom2nifti(fn, "./tmp/FLAIR/")      
    for b in mri_types:
        path=join(train_path,c)
        file=[]
        for each in  [Path(join('./tmp/',b))]:
            file.append(each.ls()[0])        
        for fn2 in file:
            pat_id = str(fn2).split('/')[-1].split('.')[0]
            if b=="FLAIR":
                final_fn = f"./test/train/"+pat_id+"_0000.nii.gz"
            if b=="T1w":
                final_fn = f"./test/train/"+pat_id+"_0001.nii.gz"
            if b=="T1wCE":
                final_fn = f"./test/train/"+pat_id+"_0002.nii.gz"    
            if b=="T2w":
                final_fn = f"./test/train/"+pat_id+"_0003.nii.gz"
            resample_nifti(fn2, ref_image, final_fn, True)
            os.remove(str(fn2))

In [ ]:
samples_test = ("../input/rsna-miccai-brain-tumor-radiogenomic-classification/test/*")
path_test=glob(samples_test)
studi_id=path_test

for c in tqdm(studi_id):
    path=join(train_path,c)
    samples = [Path(c)]
    path_train_t2w, path_train_t1wce,path_train_t1w,path_train_flair = [],[],[],[]
    for each in samples:
        path_train_t2w.append(each.ls()[0])
        path_train_t1wce.append(each.ls()[1])
        path_train_t1w.append(each.ls()[2])
        path_train_flair.append(each.ls()[3])
    for fn in path_train_t1w: dicom2nifti(fn, "./tmp/T1w/")
    for fn in path_train_t1wce: dicom2nifti(fn, "./tmp/T1wCE/")
    for fn in path_train_t1w: dicom2nifti(fn, "./tmp/T2w/")
    for fn in path_train_flair: dicom2nifti(fn, "./tmp/FLAIR/")      
    for b in mri_types:
        path=join(train_path,c)
        file=[]
        for each in  [Path(join('./tmp/',b))]:
            file.append(each.ls()[0])        
        for fn2 in file:
            pat_id = str(fn2).split('/')[-1].split('.')[0]
            if b=="FLAIR":
                final_fn = f"./test/test/"+pat_id+"_0000.nii.gz"
            if b=="T1w":
                final_fn = f"./test/test/"+pat_id+"_0001.nii.gz"
            if b=="T1wCE":
                final_fn = f"./test/test/"+pat_id+"_0002.nii.gz"    
            if b=="T2w":
                final_fn = f"./test/test/"+pat_id+"_0003.nii.gz"
            resample_nifti(fn2, ref_image, final_fn, True)
            os.remove(str(fn2))            

In [ ]:
"""
!ls ./test/post/
path_raw="./test/post/00003_0002.nii.gz"
plt.figure(figsize=(12,6))
plt.subplot(121)
flair_nib = nib.load(path_raw)
flair_nib_array = flair_nib.get_fdata()
plt.imshow(flair_nib_array[:,:,flair_nib_array.shape[2]//2])
print(flair_nib_array.shape)
path_raw="./test/post/00003_0003.nii.gz"
plt.subplot(122)
flair_nib = nib.load(path_raw)
flair_nib_array = flair_nib.get_fdata()
plt.imshow(flair_nib_array[:,:,flair_nib_array.shape[2]//2])
flair_nib_array.shape
"""

- I do the predictions for 2d and 3d_fullres, and show the result, that it is not so good, and this is the main problem, I don't know if this is registration problem, or other kind of problem.

In [ ]:
"""
!mkdir -p ./RESULTS_FOLDER/nnUNet/3d_fullres_train
!mkdir -p ./RESULTS_FOLDER/nnUNet/3d_fullres_test

!nnUNet_predict -i ./test/train/ -o ./RESULTS_FOLDER/nnUNet/3d_fullres_train -t 001 -tr nnUNetTrainerV2 -m 3d_fullres --disable_tta 
!nnUNet_predict -i ./test/test/ -o ./RESULTS_FOLDER/nnUNet/3d_fullres_test -t 001 -tr nnUNetTrainerV2 -m 3d_fullres --disable_tta 

!rm -Rf ./test
!rm -Rf ./tmp
!rm -Rf ./nnUNet_raw_data_base
!rm -Rf ./nnUNet_preprocessed
"""

In [ ]:
!mkdir -p ./RESULTS_FOLDER/nnUNet/2d
!mkdir -p ./RESULTS_FOLDER/nnUNet/2d

!nnUNet_predict -i ./test/train/ -o ./RESULTS_FOLDER/nnUNet/2d -t 001 -tr nnUNetTrainerV2 -m 2d --disable_tta 
!nnUNet_predict -i ./test/test/ -o ./RESULTS_FOLDER/nnUNet/2d -t 001 -tr nnUNetTrainerV2 -m 2d --disable_tta 

!rm -Rf ./test
!rm -Rf ./tmp
!rm -Rf ./nnUNet_raw_data_base
!rm -Rf ./nnUNet_preprocessed

In [ ]:
#!ls ./RESULTS_FOLDER/nnUNet/2d/

In [ ]:
#!pip install nibabel
"""
path_raw="./RESULTS_FOLDER/nnUNet/2d/00003.nii.gz"
flair_nib = nib.load(path_raw)
flair_nib_array = flair_nib.get_fdata()
!mkdir /tmp
import nibabel as nib
for a in range(flair_nib_array.shape[2]):
    affine = np.eye(4)
    nifti_file = nib.Nifti1Image(flair_nib_array[:,:,a], affine)
    nib.save(nifti_file, "./tmp/tmp/"+str(a))
"""

In [ ]:
"""
from matplotlib import animation, rc
rc('animation', html='jshtml')
def create_animation(ims):
    fig = plt.figure(figsize=(6, 6))
    plt.axis('off')
    im = plt.imshow(ims[0], cmap="gray")

    def animate_func(i):
        im.set_array(ims[i])
        return [im]

    return animation.FuncAnimation(fig, animate_func, frames = len(ims), interval = 1000//24)

lista=[]
for a in range(0,154):
    lista.append("./tmp/"+str(a)+".nii")

print(lista[0])
"""
#create_animation(lista)

In [ ]:
def get_array(fn):
    "opens .nii file and return the array"
    img = sitk.ReadImage(str(fn))
    imgd = sitk.GetArrayFromImage(img)
    return imgd

def plot_slice(imgd, sli):
    "given an image of shape slices x height x width, plots a slice"
    plt.imshow(imgd[sli], cmap='gray')
    plt.axis('off')
    
def get_array_plot(fn, sli):
    imgd = get_array(fn)
    plot_slice(imgd, sli)

get_array_plot(f'./RESULTS_FOLDER/nnUNet/3d_fullres/00003.nii.gz', 105)

# 4.- How to train your own model with BRATs'21 dataset

- **If you have interest and home machines in order to train the Brats'21 model, you can use the next code in order to create the specific configurations with nnUnet (https://github.com/MIC-DKFZ/nnUNet/blob/master/documentation/dataset_conversion.md):**

I did for you and I share my dataset with you, please if it is hepfull upvote: https://www.kaggle.com/victorfernandezalbor/brats2021dataset

In [ ]:
path_raw_brain="../input/brats2021dataset/nnUNet_raw_data_base/nnUNet_raw_data/Task101_BrainTumour/"
path_tr="imagesTr/"
path_ts="imagesTs/"
path_lbr="labelsTr/"
path_lbs="labelsTs/"
path_raw_brain="nnUNet_raw_data_base/nnUNet_raw_data/Task101_BrainTumour/"
task="Task101_BrainTumour"

#Uncomment and try the next code by yourself......

#!ln -rs ../input/brats2021dataset/nnUNet_raw_data_base/nnUNet_raw_data/Task101_BrainTumour/* ./nnUNet_raw_data_base/nnUNet_raw_data/Task101_BrainTumour/

##The next code generate one json which the nnUnet software use for know the test and train data, in my own dataset 
##I've added the propper one, so maybe it is not usefull for you, but it is necessary.
## Please feel free if you miss something to ask me.

#sys.path.insert(1,'../input/generatejsonbrats/')
#sys.path.insert(1,'../input/nnunet-batchgenerators/batchgenerators/')
#from batchgenerators.utilities.file_and_folder_operations import *
#from batchgenerators import *
#import file_and_folder_operations
#from utils import generate_dataset_json
#generate_dataset_json(join(path_raw_brain,'dataset.json'), path_raw_brain+path_tr, path_raw_brain+path_ts, ("flair", "t1", "t1ce", "t2"),
#                    labels={"0": "background", "1": "necrotic tumor core","2": "peritumoral edematous/invaded tissue","3":"", "4": "GD-enhancing tumor"} ,dataset_name=task, license='hands off!')


- Uncoment the next lines for running, first the preprocess, which make some augmentation and proper preprocessing in the images, and then for training the 2d and 3d_fullres configuration, with 4 folds.

More info, in the web page of the project: https://github.com/MIC-DKFZ/nnUNet#model-training

In [ ]:
#!nnUNet_plan_and_preprocess -t 101 -tf 1 --verify_dataset_integrity

In [ ]:
#!nnUNet_train 2d nnUNetTrainerV2 101 4 --npz

In [ ]:
#nnUNet_train 3d_fullres nnUNetTrainerV2 TaskXXX_MYTASK FOLD --npz
#!nnUNet_train 3d_fullres nnUNetTrainerV2 101 4 --npz